### Stand-alone code to create hexagons

With requirements needed for the spider code

In [2]:
from typing import cast

import geopandas as gpd ##requires geopandas >0.10
import h3pandas  # NoQA  type: ignore
import numpy as np
import pandas as pd
from geopandas.tools import sjoin

In [3]:
def create_hex(aoi: gpd.GeoDataFrame, resolution: int) -> gpd.GeoDataFrame:
    geom = aoi.h3.polyfill_resample(resolution).get(["geometry"])
    geom = geom.assign(h3_index=geom.index)
    geom = geom.reset_index(drop=True)
    return geom

In [4]:
def add_neighbors(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    orig_crs = gdf.crs
    gdf = cast(gpd.GeoDataFrame, gdf.to_crs(epsg=3857))
    nei = []
    for idx, row in gdf.iterrows():
        out = (
            gpd.GeoDataFrame(geometry=[row.geometry], crs=gdf.crs)
            .sjoin_nearest(gdf, how="left", max_distance=0.1)
            .index_right
        )
        out = out.loc[out != idx]
        out = np.pad(out, (0, 6 - len(out)))
        nei.append(out)

    nei = pd.DataFrame(nei, columns=[f"n{i}" for i in range(6)], index=gdf.index)  # type: ignore
    gdf = pd.concat((gdf, nei), axis=1).to_crs(orig_crs)
    return gdf

In [5]:
def feat(aoi: gpd.GeoDataFrame, hex_res: int) -> gpd.GeoDataFrame:
    geom = create_hex(aoi, hex_res)
    geom = add_neighbors(geom)
    geom["index"] = geom.index
    geom = cast(gpd.GeoDataFrame, geom.dropna(axis=0, subset=["geometry"]))
    return geom

In [4]:
# Example Usage
aoi = gpd.read_file(r"C:\Users\alexl\Dropbox\Self-employment\Imperial work\Zambia\GIS\admin\Copperbelt.geojson")

hex = feat(aoi, 6)
hex.to_file("hex.geojson")
print("Use hexagons in the spider code as instructed")